互相关运算:从MLP怎么过渡到卷积的？

In [26]:
import torch
from torch import nn as nn
from d2l import torch as d2l

def corr2d(X, K):  
    """计算二维互相关运算。"""
    h, w = K.shape # 卷积核的高和宽
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 输出的形状
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y
    

验证上述二维互相关运算的输出

In [9]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [11]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

卷积层的一个简单应用： 检测图像中不同颜色的边缘

In [12]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [13]:
K = torch.tensor([[1.0, -1.0]])

输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [14]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只可以检测垂直边缘

In [16]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由X生成Y的卷积核

In [24]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False) # 第一个1为通道数， 第二个参数是batch_size

X = X.reshape((1, 1, 6, 8)) # [通道数, batch_size, 宽, 高]
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X) # 输出
    l = (Y_hat - Y)**2 # 计算误差
    conv2d.zero_grad() # 梯度清零
    l.sum().backward() # 反向传播
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad # 更新梯度
    if (i+1) % 2:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 1, loss 27.614
batch 3, loss 8.462
batch 5, loss 2.988
batch 7, loss 1.144
batch 9, loss 0.455


所学的卷积核的权重张量

In [25]:
conv2d.weight.data

tensor([[[[ 1.0437, -0.9334]]]])